In [ ]:
import numpy as np
import pandas as pd


data_path="/home/apoorva/Documents/kaggle/movie_reviews/train.tsv"

train = pd.read_csv(data_path, header=0, \
                    delimiter="\t", quoting=3)


print(train)

train.head()

train.shape


trainfull=train['Phrase'].values
print (trainfull)

trainfull.reshape(156060,1)
print(trainfull[0])
raw_train=trainfull[1:128001]
print (raw_train.shape)
test_phrase=trainfull[128001:156033]
print(test_phrase)
print(test_phrase.shape)

Sentiment_trainfull1=train['Sentiment'].values
Sentiment_trainfull=np.zeros([156060,5])
print (Sentiment_trainfull)

for i in range(0,156059):
    a=Sentiment_trainfull1[i]
    Sentiment_trainfull[i][a]=1
print (Sentiment_trainfull)
    
    

Sentiment_train=Sentiment_trainfull[1:128001]
Sentiment_test=Sentiment_trainfull[128001:156033]

from bs4 import BeautifulSoup
import re
import nltk
#from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


#stop_words = set(stopwords.words('english'))


def clean_words(raw_phrase):
    raw_phrase_notag=BeautifulSoup(raw_phrase,'lxml').get_text()
    raw_phrase_letter=re.sub("[^a-zA-Z]"," ",raw_phrase_notag)
    raw_phrase_nocaps=raw_phrase_letter.lower()
    raw_phrase_tokens=word_tokenize(raw_phrase_nocaps)
    #raw_phrase_nostopword=[word for word in raw_phrase_tokens if word not in stop_words]
         
    return (" ".join(raw_phrase_tokens))
def count_seqlen(phrase):
    phrase_tokens=word_tokenize(phrase)
    return (len(phrase_tokens))
    

clean_train=[]
seqlen=[]
train_seqlen=[]
size=raw_train.size
for i in range(0,size):
    
    clean_train.append(clean_words(raw_train[i]))
    seqlen.append(count_seqlen(clean_train[i]))
    train_seqlen.append(count_seqlen(clean_train[i]))

   
print (clean_train[6767])   
print (seqlen[6767])

sizetest=test_phrase.size
test_seqlen=[]
for i in range(0,sizetest):
    test_seqlen.append(count_seqlen(test_phrase[i]))
max_seqlen_train=np.amax(test_seqlen)
max_seqlen_test=np.amax(test_seqlen)
if max_seqlen_train>max_seqlen_test:
    max_seqlen=max_seqlen_train
else:
    max_seqlen=max_seqlen_test
print (max_seqlen)

wordsList = np.load('../input/wordsList.npy')

wordsList = wordsList.tolist() #Originally loaded as numpy array
wordsList = [word.decode('UTF-8') for word in wordsList] #Encode words as UTF-8
wordVectors = np.load('../input/wordVectors.npy')

def create_id_matrix(clean_train,size,max_seqlen):
    id_matrix=np.zeros((size,max_seqlen),dtype='int32')
    for phrase_count in range(0,size):
        tokens=word_tokenize(clean_train[phrase_count])
        word_count=0
        for word in tokens:
            try:
                id_matrix[phrase_count][word_count]=wordsList.index(word)
            except ValueError:
                id_matrix[phrase_count][word_count] = 399999
            word_count=word_count+1
    return (id_matrix)
            

size_test=test_phrase.size


id_matrix=create_id_matrix(clean_train,size,max_seqlen)


print (id_matrix)
id_matrix=np.asarray(id_matrix)
print (id_matrix)

id_matrix_test=create_id_matrix(test_phrase,size_test,max_seqlen)


print(id_matrix.shape)

#id_matrix_train=id_matrix[1:62424]
#id_matrix_validate=id_matrix[62425:78029]

#print (id_matrix_train.shape)

#print (id_matrix_test.shape)

#print (id_matrix_validate.shape)

np.save('idsMatrix1', id_matrix)
np.save('idsMatrix', id_matrix_test)

id_matrix=np.load('idsMatrix1.npy')
id_matrix_test=np.load('idsMatrix.npy')

id_matrix[6767][1]

#id_matrix_train=id_matrix[1:78030]
#id_matrix_test=id_matrix[78030:]

batchSize = 64
lstmUnits = 64
numClasses = 5

max_seqlen_test=np.amax(test_seqlen)

import tensorflow as tf

tf.reset_default_graph()

labels = tf.placeholder(tf.float32, [batchSize, numClasses])
input_data = tf.placeholder(tf.int32, [batchSize, max_seqlen_test])

inp=tf.placeholder(tf.int32,[28059,max_seqlen_test])
out=tf.placeholder(tf.float32,[28059,numClasses])

print (wordVectors.shape)
len(wordsList)

numDimensions=50

data = tf.Variable(tf.zeros([batchSize, max_seqlen, numDimensions]))

data = tf.nn.embedding_lookup(wordVectors,input_data)

lstmCell = tf.contrib.rnn.BasicLSTMCell(lstmUnits)

lstmCell = tf.contrib.rnn.DropoutWrapper(cell=lstmCell, output_keep_prob=0.75)

value, _ = tf.nn.dynamic_rnn(lstmCell, data, dtype=tf.float32)

weight = tf.Variable(tf.truncated_normal([lstmUnits, numClasses]))
bias = tf.Variable(tf.constant(0.1, shape=[numClasses]))
value = tf.transpose(value, [1, 0, 2])
last = tf.gather(value, int(value.get_shape()[0]) - 1)
prediction = (tf.matmul(last, weight) + bias)

correctPred = tf.equal(tf.argmax(prediction,1), tf.argmax(labels,1))
accuracy = tf.reduce_mean(tf.cast(correctPred, tf.float32))

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=labels))
optimizer = tf.train.AdamOptimizer().minimize(loss)


def Trainbatch(i,batch_size,inp,out):
    #print (i)
    a=(i-1)*batch_size
    #print (a)
    b =(i*batch_size)
    #print(b)
    y=tf.zeros([batch_size,numClasses])
    x=tf.zeros([batch_size,max_seqlen])
    x=inp[a:b]
    #print (x.shape)
    y=out[a:b]
    #print (y.shape)
    #print (y)
    return x,y



sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
iterations = 1000
counter=0
#128000/64(batchsiz)=2000
for i in range(iterations):
    for j in range(1,2000):
    
        nextBatch, nextBatchLabels= Trainbatch(j,batchSize,id_matrix,Sentiment_train);
        sess.run(optimizer, {input_data: nextBatch, labels: nextBatchLabels})
        print(i,j)
   


'''#def getValidateBatch(counter):
#    labels=np.zeros([batchSize,numClasses])
 #   arr = np.zeros([batchSize, max_seqlen])
  #  for i in range(counter,(counter+batchSize)):
   #     arr[i] = id_matrix_validate[i]
    #    value=Sentiment_test[i]
     #   labels[i][value-1]=1
      #  counter=counter+1
    #return arr, labels,counter
    
    
def Trainbatch(i,batch_size,inp,out):
    #print (i)
    a=(i-1)*batch_size
    #print (a)
    b =(i*batch_size)
    #print(b)
    x=tf.zeros([batch_size,3])
    y=tf.zeros([batch_size,1])
    x=inp[a:b]
    #print (x.shape)
    y=out[a:b]
    #print (y.shape)
    return x,y
'''

def Testbatch(i,batch_size,inp,out):
    #print (i)
    a=(i-1)*batch_size
    #print (a)
    b =(i*batch_size)
    #print(b)
    y=tf.zeros([batch_size,numClasses])
    x=tf.zeros([batch_size,max_seqlen])
    x=inp[a:b]
    #print (x.shape)
    y=out[a:b]
    #print (y.shape)
    #print (y)
    return x,y


iteration = 5
total=0
for i in range(iteration):
    for j in range(1,438):
          
        nextBatch, nextBatchLabels = Testbatch(j,batchSize,id_matrix_test,Sentiment_test)
        acc=(sess.run(accuracy, {input_data: nextBatch, labels: nextBatchLabels})) * 100
        total=(total+acc)
        print (i,j)
        total=total/438
    print("Accuracy for this batch:", total)

    
    